# Úkol č. 3 - Segmentace zákazníků e-shopu
**Deadline úkolu je uveden na [course pages](https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html).**

Jednou z důležitých aplikací shlukování je **segmentace zákazníků** (angl. **customer segmentation**). 

Předpokládejme, že máme následující obchodní údaje o prodejích (resp. nákupech z pohledu zákazníků):
TransactionID - ID nákupu,
CustomerID - ID zákazníka, 
Date - datum nákupu, 
Total - celková cena nákupu.

Chceme najít segmenty zákazníků, kteří se chovají podobně. K tomu je dobré informace z jednotlivých nákupů pro individuální zákazníky agregovat. Tj. získat pro každého zákazníka jeden řádek.

Populárním přístupem je **RFM**, což znamená:

- **R**ecency: Počet dnů od posledního nákupu (poslední datum v datasetu pro daného zákazníka).
- **F**requency: Počet nákupů. Občas se vynechávají zákazníci s jediným nákupem. Pro jednoduchost je zde ale necháme.
- **M**onetary: Celková suma, kterou daný zákazník utratil.

## Zdroj dat
Budeme pracovat s daty z jednoho (skoro) vymyšleného eshopu, která jsou v přiloženém souboru `eshop.csv`.

## Pokyny k vypracování

Ve všech bodech zadání uvažujte aktuální datum jako datum poslední transakce (19. 12. 2015), nikoliv dnešek. Tváříme se, že jde o aktuální data.

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
- Vytvořte `rfm` data frame, kde každý řádek odpovídá jednomu zákazníkovi a sloupce (příznaky) jsou uvedené výše.
- Pomocí algoritmu `K-means` proveďte shlukování. Nějakým způsobem také odhadněte nejlepší počet shluků (podrobně vysvětlete).
- Zabývejte se vlivem přeškálování dat (standardizace příznaků). Tj. určete, zda je přeškálování vhodné, a proveďte ho.
- Interpretujte jednotlivé shluky. Použijte získané shluky k odlišení "superstar" zákazníků (vysoká monetary, vysoká frequency a nízká recency) od nezajímavých  zákazníků (vysoká recency, nízká frequency, nízká monetary).

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
- (až +4 body) Proveďte analýzu vytvořených shluků pomocí metody silhouette (https://en.wikipedia.org/wiki/Silhouette_(clustering)).
- (až +4 body) Zkuste provést to samé s modifikovanou verzí **RFM**, kde Recency = "maximum počtu měsíců od posledního nákupu a čísla 1", Frequency = "maximum počtu nákupů daného zákazníka v posledních 12 měsících a čísla 1", Monetary = "Nejvyšší hodnota nákupu daného zákazníka". Porovnejte s původním přístupem.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte Jupyter Notebook.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

# -----------------------------------------------------------------------------------------------------------

## Imports

In [215]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Preprocess data

In [216]:
data = pd.read_csv('eshop.csv')
data.head(5)

,Customer ID,Date,Subtotal,Country
0,34,6/21/2007,86.0,Czech Republic
1,38,6/27/2007,38.4,Czech Republic
2,47,7/2/2007,53.5,Slovakia
3,61,7/14/2007,7.0,Czech Republic
4,78,7/21/2007,55.5,Czech Republic


In [217]:
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values('Date', ascending=False, inplace=True)

last_transaction = data.iloc[0]['Date']
print(f'Last recorded transaction took place on: {last_transaction}')

Last recorded transaction took place on: 2015-12-19 00:00:00


In [218]:
rfm = data.drop(['Subtotal', 'Country'], axis=1)

### Recency

In [219]:
rfm = rfm.drop_duplicates(subset='Customer ID', keep='first')
rfm['Recency'] = rfm['Date'].apply(lambda x: (last_transaction - x).days)
#rfm['Recency'] = rfm['Date'].apply(lambda x: math.log((last_transaction - x).days + 1))
rfm = rfm.drop(['Date'], axis=1)

,Customer ID,Recency
22407,15360,0
22406,15359,0
22399,15354,1
22394,15350,1
22395,15351,1
...,...,...
5,79,3073
3,61,3080
2,47,3092
1,38,3097


### Frequency

In [220]:
counts = data.groupby('Customer ID').count()['Date'].rename('Frequency')
rfm = rfm.merge(counts, how='left', on='Customer ID')

,Customer ID,Recency,Frequency
0,15360,0,1
8180,7115,1382,1
8181,7096,1383,1
8182,7097,1383,1
8183,7098,1383,1
...,...,...,...
84,4667,9,27
10336,2074,1830,28
4854,5291,775,28
90,2482,9,32


### Monetary

In [221]:
totals = data.groupby('Customer ID')['Subtotal'].sum().rename('Monetary')
rfm = rfm.merge(totals, how='left', on='Customer ID')
rfm.sort_values('Monetary')

,Customer ID,Recency,Frequency,Monetary
4453,11428,703,1,1.18
7429,8037,1237,1,1.64
11451,2867,2075,1,1.68
11775,2398,2175,1,1.80
9475,5442,1650,1,2.05
...,...,...,...,...
1853,12294,240,3,7851.53
2440,13481,341,2,8739.21
2406,9639,337,24,8945.18
5912,6544,969,7,11427.92


In [260]:
# source to function: https://www.xspdf.com/help/50899244.html

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+= df.to_html() + '&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp'
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [262]:
display_side_by_side(rfm.sort_values('Recency').tail(15)[['Customer ID', 'Recency']],
                     rfm.sort_values('Frequency').tail(15)[['Customer ID', 'Frequency']],
                     rfm.sort_values('Monetary').tail(15)[['Customer ID', 'Monetary']])

,Customer ID,Recency
12814,166,3013
12815,30,3025
12816,131,3033
12817,96,3042
12818,67,3043
12819,9,3050
12820,62,3054
12821,82,3069
12822,48,3069
12823,78,3073
